In [1]:
import os
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt
from tifffile import imwrite
from PIL import Image
from tqdm import tqdm
from copy import deepcopy
import json
from skimage.color import gray2rgb


In [2]:
p = "../paths.json"
with open(p, "r") as f:
    d = json.load(f)

In [3]:
d

{'0011284': {'img': '/projectbig/jupyternotebook/xenium_cgn/robin_xenium_cgn_from_epyc/original_TIF/converted_OME/0011284.ome.tif',
  'adata': '/projectbig/jupyternotebook/xenium_cgn/robin_xenium_cgn_from_epyc/h5ad_aligned/0011284.h5ad'},
 '0011546': {'img': '/projectbig/jupyternotebook/xenium_cgn/robin_xenium_cgn_from_epyc/original_TIF/converted_OME/0011546.ome.tif',
  'adata': '/projectbig/jupyternotebook/xenium_cgn/robin_xenium_cgn_from_epyc/h5ad_aligned/0011546.h5ad'},
 '0011216': {'img': '/projectbig/jupyternotebook/xenium_cgn/robin_xenium_cgn_from_epyc/original_TIF/converted_OME/0011216.ome.tif',
  'adata': '/projectbig/jupyternotebook/xenium_cgn/robin_xenium_cgn_from_epyc/h5ad_aligned/0011216.h5ad'},
 '0011707': {'img': '/projectbig/jupyternotebook/xenium_cgn/robin_xenium_cgn_from_epyc/original_TIF/converted_OME/0011707.ome.tif',
  'adata': '/projectbig/jupyternotebook/xenium_cgn/robin_xenium_cgn_from_epyc/h5ad_aligned/0011707.h5ad'},
 '0011287': {'img': '/projectbig/jupyternote

In [4]:
LEVEL = 1

In [5]:
adata = sc.read(d[f"combined_LEVEL{LEVEL}"]["adata"])
# adata = sc.read('/projectbig/jupyternotebook/xenium_cgn/robin_xenium_cgn_from_epyc/h5ad_aligned/merged_with_Disease_LEVEL0.h5ad')

In [6]:
adata.obs["Slide_ID"].value_counts()

Slide_ID
0011216    445832
0011287    407779
0011695    392638
0011546    370042
0011707    356653
0018775    325998
0011284    321855
0011762    278382
Name: count, dtype: int64

In [7]:
savedir = "./"
savedir = "/projectbig/jupyternotebook/xenium_cgn/robin_xenium_cgn_from_epyc/seperate_samples/"

In [8]:
savedir_adata = os.path.join(savedir, f"adata_LEVEL{LEVEL}_new")
if not os.path.exists(savedir_adata):
    os.mkdir(savedir_adata)

savedir_imgs = os.path.join(savedir, f"imgs_LEVEL{LEVEL}_new")
if not os.path.exists(savedir_imgs):
    os.mkdir(savedir_imgs)

savedir_adata, savedir_imgs

('/projectbig/jupyternotebook/xenium_cgn/robin_xenium_cgn_from_epyc/seperate_samples/adata_LEVEL1_new',
 '/projectbig/jupyternotebook/xenium_cgn/robin_xenium_cgn_from_epyc/seperate_samples/imgs_LEVEL1_new')

In [9]:
samples = adata.obs.Sample_num.unique()

In [10]:
IDs = adata.obs.Slide_ID.unique()

In [11]:
def cap(val, index = 0):
    if val < 0:
        val = 0
    # elif val > img.shape[index]:
    #     val = img.shape[index]
    return val

In [ ]:
for sample in tqdm(adata.obs.ID.unique()):
    
    sub = adata[adata.obs.ID==sample]
    ID = sub.obs.Slide_ID.unique()[0]
    scale_factor = sub.uns["spatial"][ID]["scalefactors"][f"tissue_dapi_LEVEL{LEVEL}_scalef"]
    #ax=sc.pl.embedding(sub, basis="spatial", s=20, show=False)
    ylim=(sub.obsm["spatial"][:,1].min()-350, sub.obsm["spatial"][:,1].max()+350)
    xlim=(sub.obsm["spatial"][:,0].min()-350, sub.obsm["spatial"][:,0].max()+350)
    ylim = tuple(np.array(ylim)*scale_factor)
    xlim = tuple(np.array(xlim)*scale_factor)
    ylim = tuple([cap(val, 1) for val in ylim])
    xlim = tuple([cap(val, 0) for val in xlim])

    #plt.show()
    #plt.clf()

    img = sub.uns["spatial"][ID]["images"][f"dapi_LEVEL{LEVEL}"]
    cur_coords = np.concatenate([xlim, ylim])
    img_cropped = img[int(cur_coords[2]): int(np.ceil(cur_coords[3])),\
                     int(cur_coords[0]): int(np.ceil(cur_coords[1]))]

    sub.uns["spatial"][ID]["images"][f"dapi_LEVEL{LEVEL}"+"_cropped"] = img_cropped
    sub.uns["spatial"][ID]["scalefactors"]["tissue_dapi_LEVEL{LEVEL}_cropped_scalef"] = 1

    sub.obsm["spatial"][:,0] = sub.obsm["spatial"][:,0]*scale_factor-int(cur_coords[0])
    sub.obsm["spatial"][:,1] = sub.obsm["spatial"][:,1]*scale_factor-int(cur_coords[2])

    keys = list(sub.uns["spatial"].keys())
    newdict = deepcopy(sub.uns["spatial"])
    for key in keys:
        if key!=ID:
            del newdict[key]
    sub.uns["spatial"] = newdict
    
    img_to_save = Image.fromarray((sub.uns["spatial"][ID]["images"][f"dapi_LEVEL{LEVEL}"+"_cropped"]*255).astype(np.uint8))
    im_arr = (sub.uns["spatial"][ID]["images"][f"dapi_LEVEL{LEVEL}"+"_cropped"]*255).astype(np.uint8)
    folder = sub.obs.Slide_ID.unique()[0]
    if not os.path.exists(os.path.join(savedir_imgs, folder)):
        os.mkdir(os.path.join(savedir_imgs, folder))
    if not os.path.exists(os.path.join(savedir_adata, folder)):
        os.mkdir(os.path.join(savedir_adata, folder))
    img_to_save.save(os.path.join(savedir_imgs, f"{sample.split('_')[1]}.png"),"PNG")
    imwrite(os.path.join(savedir_imgs, f"{sample.split('_')[1]}.tiff"), im_arr)
    sub.write(os.path.join(savedir_adata, f"{sample.split('_')[1]}.h5ad"))

  0%|          | 0/63 [00:00<?, ?it/s]/tmp/ipykernel_2037/3389823328.py:25: ImplicitModificationWarning: Trying to modify attribute `.obsm` of view, initializing view as actual.
  sub.obsm["spatial"][:,0] = sub.obsm["spatial"][:,0]*scale_factor-int(cur_coords[0])
  2%|▏         | 1/63 [03:36<3:43:34, 216.36s/it]/tmp/ipykernel_2037/3389823328.py:25: ImplicitModificationWarning: Trying to modify attribute `.obsm` of view, initializing view as actual.
  sub.obsm["spatial"][:,0] = sub.obsm["spatial"][:,0]*scale_factor-int(cur_coords[0])
  3%|▎         | 2/63 [07:36<3:54:07, 230.28s/it]/tmp/ipykernel_2037/3389823328.py:25: ImplicitModificationWarning: Trying to modify attribute `.obsm` of view, initializing view as actual.
  sub.obsm["spatial"][:,0] = sub.obsm["spatial"][:,0]*scale_factor-int(cur_coords[0])
  5%|▍         | 3/63 [12:03<4:07:15, 247.25s/it]/tmp/ipykernel_2037/3389823328.py:25: ImplicitModificationWarning: Trying to modify attribute `.obsm` of view, initializing view as actua